In [1]:
import nbimporter
import pandas as pd
import numpy as np
import awswrangler as wr
import math
import os
import boto3
from os import listdir
from os.path import isfile, join

from cliofunctions import ac_aws, normalize, sort_and_group, operation, extract_info, porcentaje, promedio, ranking, top

# CONAF
El siguiente script extrae los datos de la cantidad de incendios por comuna durante los años 2016 al 2019 declarados por la CONAF. Notar que al imprimir el archivo, la columna que tiene los nombres de las comunas tiene el nombre de "COMUNA", es importante definirlo como "main_groupe".

In [2]:
main_df = pd.read_csv("CONAF.csv",sep=";")
main_groupe = "COMUNA"
main_df.head()

,COMUNA,TOTAL INCENDIOS 2019,TOTAL INCENDIOS 2018,TOTAL INCENDIOS 2017,TOTAL INCENDIOS 2016
0,ARICA,4,4.0,5.0,1.0
1,CAMARONES,2,2.0,1.0,NaN
2,PUTRE,2,2.0,NaN,NaN
3,GENERAL LAGOS,0,NaN,NaN,NaN
4,IQUIQUE,0,NaN,NaN,NaN


### Matriz de consulta
La lectura del matriz de consulta se realiza desde el servidor AWS

In [3]:
matriz_c = "s3://test1-rvc/Matriz_consulta.csv"
matrix_df = pd.read_csv(matriz_c, sep=";")
matrix_df.head()

,DIMENSIÓN,SUBDIMENSIÓN,VARIABLE,INDICADOR,DESCRIPCIÓN,NOMBRE VARIABLE BBDD,FUENTE,AÑO,ALCANCE,IQUIQUE,...,RANQUIL,TREGUACO,SAN CARLOS,COIHUECO,QUILLON,SAN IGNACIO,YUNGAY,QUIRIHUE,COBQUECURA,COELEMU
0,CARACTERIZACIÓN BÁSICA,MACROS DEMOGRÁFICOS,IDENTIFICACIÓN TERRITORIAL,CÓDIGO ÚNICO TERRITORIAL,CÓDIGO ÚNICO QUE IDENTIFICA CADA COMUNA,COMUNA,CENSO,2017,COMUNAL,1101,...,16206,16207,16301,16302,16107,16108,16109,16201,16202,16203
1,CARACTERIZACIÓN BÁSICA,MACROS DEMOGRÁFICOS,POBLACIÓN,POBLACIÓN,CANTIDAD DE PERSONAS TOTALES,P08,CENSO PERSONAS,2017,COMUNAL,191468,...,5755,5401,53024,26881,17485,16079,17787,11594,5012,15995
2,CARACTERIZACIÓN BÁSICA,POBLACIÓN ORIGINARIA,PERTENENCIA,N° DE POBLACIÓN QUE SE SIENTE PERTENECIENTE,PERTENECIA A PUEBLO ORIGINARIO (PRIMERA MAYORIA),r3,CASEN,2020,COMUNAL,Mapuche,...,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,
3,CARACTERIZACIÓN BÁSICA,POBLACIÓN ORIGINARIA,PERTENENCIA,N° DE POBLACIÓN QUE SE SIENTE PERTENECIENTE,PERTENECIA A PUEBLO ORIGINARIO (SEGUNDA MAYORIA),r3,CASEN,2020,COMUNAL,Diaguita,...,,,,,,Diaguita,,,,
4,CARACTERIZACIÓN BÁSICA,POBLACIÓN ORIGINARIA,PERTENENCIA,N° DE POBLACIÓN QUE SE SIENTE PERTENECIENTE,PERTENECIA A PUEBLO ORIGINARIO (TERCERA MAYORIA),r3,CASEN,2020,COMUNAL,Quechua,...,,,,,,,,,,


### Archivos complementarios
CSV que indican orden de comunas, de región o excepciones para reemplazar en los nombres de las comunas y/o regiones de cada dataset. Notar que en este caso el archivo tiene un alcance COMUNAL.

In [4]:
sort_by_comuna = "s3://test1-rvc/Sort_by_comuna.csv"
sort_by_region = "s3://test1-rvc/Sort_by_region.csv"
replace_sort = "s3://test1-rvc/Sort_replace_weird_objects.csv"

#reading dataframe de orgen (regional o comunal) y matriz de reemplazos (para singularidades en comunas o regiones)
reach = "COMUNAL"
if reach == "COMUNAL":
    sort_df = pd.read_csv(sort_by_comuna, sep = ";")
elif reach == "REGIONAL":
    sort_df = pd.read_csv(sort_by_region, sep = ";")
else:
    print("Debe indicar alcance: REGIONAL O COMUNAL")
replace = pd.read_csv(replace_sort, sep = ";")

### Descriptor de variables

In [5]:
#reading variables of interest
descriptor =  pd.read_excel("Descriptor.xlsx")
descriptor

,DIMENSIÓN,SUBDIMENSIÓN,VARIABLE,INDICADOR,DESCRIPCIÓN,NOMBRE VARIABLE BBDD,FUENTE,AÑO,ALCANCE
0,SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2019,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2019,COMUNAL
1,SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2018,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2018,COMUNAL
2,SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2017,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2017,COMUNAL
3,SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2016,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2016,COMUNAL


### Juntar el descriptor con los datos

In [6]:
main_df = sort_and_group(main_df, main_groupe, sort_df, replace).reset_index()
df_add = pd.concat((descriptor,main_df),axis=1)
del df_add["index"]
df_add

,DIMENSIÓN,SUBDIMENSIÓN,VARIABLE,INDICADOR,DESCRIPCIÓN,NOMBRE VARIABLE BBDD,FUENTE,AÑO,ALCANCE,1101,...,16206,16207,16301,16302,16107,16108,16109,16201,16202,16203
0,SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2019,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2019,COMUNAL,0.0,...,21.0,8.0,26.0,77.0,32.0,18.0,30.0,15.0,10.0,54.0
1,SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2018,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2018,COMUNAL,,...,33.0,8.0,45.0,57.0,57.0,11.0,13.0,16.0,3.0,39.0
2,SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2017,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2017,COMUNAL,,...,4.0,10.0,7.0,7.0,54.0,6.0,5.0,34.0,17.0,15.0
3,SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2016,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2016,COMUNAL,,...,4.0,13.0,8.0,2.0,45.0,7.0,11.0,43.0,29.0,14.0


In [7]:
df_add.columns = matrix_df.columns
matrix_df_act = pd.concat((matrix_df,df_add),axis=0).set_index("DIMENSIÓN")
matrix_df_act

,SUBDIMENSIÓN,VARIABLE,INDICADOR,DESCRIPCIÓN,NOMBRE VARIABLE BBDD,FUENTE,AÑO,ALCANCE,IQUIQUE,ALTO HOSPICIO,...,RANQUIL,TREGUACO,SAN CARLOS,COIHUECO,QUILLON,SAN IGNACIO,YUNGAY,QUIRIHUE,COBQUECURA,COELEMU
DIMENSIÓN,,,,,,,,,,,,,,,,,,,,,
CARACTERIZACIÓN BÁSICA,MACROS DEMOGRÁFICOS,IDENTIFICACIÓN TERRITORIAL,CÓDIGO ÚNICO TERRITORIAL,CÓDIGO ÚNICO QUE IDENTIFICA CADA COMUNA,COMUNA,CENSO,2017,COMUNAL,1101,1107,...,16206,16207,16301,16302,16107,16108,16109,16201,16202,16203
CARACTERIZACIÓN BÁSICA,MACROS DEMOGRÁFICOS,POBLACIÓN,POBLACIÓN,CANTIDAD DE PERSONAS TOTALES,P08,CENSO PERSONAS,2017,COMUNAL,191468,108375,...,5755,5401,53024,26881,17485,16079,17787,11594,5012,15995
CARACTERIZACIÓN BÁSICA,POBLACIÓN ORIGINARIA,PERTENENCIA,N° DE POBLACIÓN QUE SE SIENTE PERTENECIENTE,PERTENECIA A PUEBLO ORIGINARIO (PRIMERA MAYORIA),r3,CASEN,2020,COMUNAL,Mapuche,Mapuche,...,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,Mapuche,
CARACTERIZACIÓN BÁSICA,POBLACIÓN ORIGINARIA,PERTENENCIA,N° DE POBLACIÓN QUE SE SIENTE PERTENECIENTE,PERTENECIA A PUEBLO ORIGINARIO (SEGUNDA MAYORIA),r3,CASEN,2020,COMUNAL,Diaguita,Diaguita,...,,,,,,Diaguita,,,,
CARACTERIZACIÓN BÁSICA,POBLACIÓN ORIGINARIA,PERTENENCIA,N° DE POBLACIÓN QUE SE SIENTE PERTENECIENTE,PERTENECIA A PUEBLO ORIGINARIO (TERCERA MAYORIA),r3,CASEN,2020,COMUNAL,Quechua,Quechua,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SOCIAL,SEGURIDAD,PERCEPCIÓN,N° DE LA POBLACIÓN QUE EN LOS ÚLTIMOS 12 MESES...,CANTIDAD DE LA POBLACIÓN QUE EN LOS ÚLTIMOS 12...,J7_3_1,ENUSC,2020,REGIONAL,5557,5557,...,,,,,,,,,,
SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2019,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2019,COMUNAL,0.0,0.0,...,21.0,8.0,26.0,77.0,32.0,18.0,30.0,15.0,10.0,54.0
SOCIAL,AMBIENTAL,INCENDIOS,INCENDIOS 2018,NÚMERO TOTAL DE INCENDIOS EN LA COMUNA EL AÑO ...,NO APLICA,CONAF,2018,COMUNAL,,,...,33.0,8.0,45.0,57.0,57.0,11.0,13.0,16.0,3.0,39.0


# Exportar
Se exporta la Matriz de consulta con los datos de CONAF agregados

In [8]:
matrix_df_act.to_csv('Matriz_consulta.csv',encoding='utf-8-sig')